## Run Anima on Google Colab via ComfyUI
* 🤗 [Anima Model On Hugging Face](https://huggingface.co/circlestone-labs/Anima)
* 📄 **License**: Provided under the [circlestone-labs-non-commercial-license
](https://huggingface.co/circlestone-labs/Anima/blob/main/LICENSE.md)

In [ ]:
#@title Install ComfyUI and Download Model
!apt -q install -y aria2
%cd /content/
!git clone https://github.com/comfyanonymous/ComfyUI
%cd /content/ComfyUI
!pip -q install -r requirements.txt
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git || true
%cd /content/ComfyUI/custom_nodes/ComfyUI-Manager
!pip install -r requirements.txt



!aria2c -c -x16 -s16 -k1M https://huggingface.co/circlestone-labs/Anima/resolve/main/split_files/diffusion_models/anima-preview.safetensors \
-d /content/ComfyUI/models/diffusion_models -o anima-preview.safetensors > /dev/null 2>&1 \
&& echo "✅ Diffusion model download complete" || echo "❌ Diffusion model download failed"

!aria2c -c -x16 -s16 -k1M https://huggingface.co/pachiiahri/anima-fp8-comfyui/resolve/main/anima-preview_tcfp8_mixed.safetensors \
-d /content/ComfyUI/models/diffusion_models -o anima-preview_tcfp8_mixed.safetensors > /dev/null 2>&1 \
&& echo "✅ fp8 Diffusion model download complete" || echo "❌ fp8 Diffusion model download failed"

!aria2c -c -x16 -s16 -k1M https://huggingface.co/circlestone-labs/Anima/resolve/main/split_files/text_encoders/qwen_3_06b_base.safetensors \
-d /content/ComfyUI/models/text_encoders -o qwen_3_06b_base.safetensors > /dev/null 2>&1 \
&& echo "✅ Text encoder download complete" || echo "❌ Text encoder download failed"

!aria2c -c -x16 -s16 -k1M https://huggingface.co/circlestone-labs/Anima/resolve/main/split_files/vae/qwen_image_vae.safetensors \
-d /content/ComfyUI/models/vae -o qwen_image_vae.safetensors > /dev/null 2>&1 \
&& echo "✅ VAE download complete" || echo "❌ VAE download failed"
import requests, json, os

url = "https://raw.githubusercontent.com/NeuralFalconYT/comfyUI_Colab/refs/heads/main/workflow/anima.json"

save_path = "/content/ComfyUI/user/default/workflows"
os.makedirs(save_path, exist_ok=True)

data = requests.get(url).json()

with open(f"{save_path}/anima.json", "w") as f:
    json.dump(data, f, indent=2)

print("✅ Workflow set as default")

from IPython.display import clear_output
clear_output()
from IPython.display import Audio,display
display(Audio("https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a", autoplay=True))

In [ ]:

#@title Run Comfy UI
%cd /content
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb >/dev/null 2>&1 || true

import subprocess, re,time ,socket,threading

def tunnel_printer(port):
    while True:
        time.sleep(0.5)
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            if s.connect_ex(("127.0.0.1", port)) == 0:
                break
        finally:
            s.close()
    print("\nComfyUI is up. Launching cloudflared tunnel...")
    p = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )
    for line in p.stderr:
        if "trycloudflare.com" in line:
            url = re.findall(r"https?://[^\s]+trycloudflare\.com[^\s]*", line)
            if url:
                print("\n🔗 Access ComfyUI:", url[0])

%cd /content/ComfyUI
PORT=8188
threading.Thread(target=tunnel_printer, args=(PORT,), daemon=True).start()

print("\nStarting ComfyUI...")
!python main.py --listen 0.0.0.0 --port {PORT} --dont-print-server
